In [ ]:

!pip install ultralytics deep_sort_realtime opencv-python-headless matplotlib numpy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00


In [ ]:

import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow 


uploaded = files.upload()
for filename in uploaded.keys():
    VIDEO_PATH = filename
print(f"Uploaded video file: {VIDEO_PATH}")

#title Configuration (Change lines here as needed)
LINE_UP = 150       # Y-coordinate of upper horizontal line
LINE_DOWN = 400     # Y-coordinate of lower horizontal line
CONF_THRESH = 0.4   # Detection confidence threshold

OUTPUT_VIDEO_PATH = "output_people_count_heatmap.mp4"
HEATMAP_PATH = "heatmap.png"

COLOR_LINE_UP = (0, 255, 0)
COLOR_LINE_DOWN = (0, 0, 255)
COLOR_IN = (0, 255, 0)
COLOR_OUT = (0, 0, 255)

PERSON_CLASS = 0

#@title Initialize models and variables
model = YOLO("yolov8n.pt") 
tracker = DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0, max_cosine_distance=0.2)

track_memory = {}  # track_id: last_center_y
count_in = 0
count_out = 0
heatmap_accum = None

# Helper functions

def draw_lines(frame):
    h, w = frame.shape[:2]
    cv2.line(frame, (0, LINE_UP), (w, LINE_UP), COLOR_LINE_UP, 2)
    cv2.line(frame, (0, LINE_DOWN), (w, LINE_DOWN), COLOR_LINE_DOWN, 2)

def update_counts(track_id, y, track_memory):
    global count_in, count_out
    if track_id in track_memory:
        prev_y = track_memory[track_id]
        if prev_y < LINE_UP and y >= LINE_UP:
            count_in += 1
            del track_memory[track_id]
            return "IN"
        elif prev_y > LINE_DOWN and y <= LINE_DOWN:
            count_out += 1
            del track_memory[track_id]
            return "OUT"
    track_memory[track_id] = y
    return None

def generate_heatmap(heatmap_accum, centers, frame_shape):
    for c in centers:
        x, y = int(c[0]), int(c[1])
        if 0 <= y < frame_shape[0] and 0 <= x < frame_shape[1]:
            heatmap_accum[y, x] += 1
    return heatmap_accum

#Run detection, tracking, counting, heatmap
def main():
    global heatmap_accum, count_in, count_out

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Cannot open video: {VIDEO_PATH}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    heatmap_accum = np.zeros((height, width), dtype=np.float32)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, conf=CONF_THRESH, classes=[PERSON_CLASS], verbose=False)

        detections = []
        centers = []

        for r in results:
            boxes = r.boxes
            for box in boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                cls = int(box.cls[0].cpu().numpy())

                if conf < CONF_THRESH or cls != PERSON_CLASS:
                    continue

                x1, y1, x2, y2 = map(int, xyxy)
                w = x2 - x1
                h = y2 - y1
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)
                centers.append((cx, cy))
                # Pass detections as a tuple: ((x1, y1, w, h), conf)
                detections.append(((x1, y1, w, h), conf))


        heatmap_accum = generate_heatmap(heatmap_accum, centers, (height, width))

        tracks = tracker.update_tracks(detections, frame=frame)

        draw_lines(frame)

        for track in tracks:
            # Check if the track is confirmed before processing
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            ltrb = track.to_ltrb()
            x1, y1, x2, y2 = map(int, ltrb)
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            result = update_counts(track_id, cy, track_memory)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

            if result == "IN":
                cv2.putText(frame, "IN", (cx, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR_IN, 3)
            elif result == "OUT":
                cv2.putText(frame, "OUT", (cx, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR_OUT, 3)

        cv2.putText(frame, f"IN: {count_in}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR_IN, 2)
        cv2.putText(frame, f"OUT: {count_out}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR_OUT, 2)

        out.write(frame)

        # Show live frame (optional, comment if slow)
        cv2_imshow(frame) # Use cv2_imshow instead of cv2.imshow
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save heatmap image
    heatmap_norm = cv2.normalize(heatmap_accum, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    heatmap_color = cv2.applyColorMap(heatmap_norm, cv2.COLORMAP_JET)
    cv2.imwrite(HEATMAP_PATH, heatmap_color)

    # Display heatmap
    plt.figure(figsize=(10, 6))
    plt.title("Heatmap of Movement Intensity")
    plt.imshow(cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    print(f"Finished! Output video saved as {OUTPUT_VIDEO_PATH}")
    print(f"Heatmap saved as {HEATMAP_PATH}")

#@title Run main
main()